In [ ]:
"""
Consider reading the 'README.md' file for more clarity on this program/code.
"""

In [ ]:
import time 
import selenium
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup as bs
wd = webdriver.Chrome(executable_path=r'C:\Users\Sravan Kumar\chromedriver.exe')
wd.maximize_window()

In [ ]:
#LinkedIn SignIn Page
wd.get('https://www.linkedin.com/login?fromSignIn=true&trk=guest_homepage-basic_nav-header-signin')
time.sleep(5)

In [ ]:
u=wd.find_element_by_name('session_key')
u.send_keys('Email Input') #Give your LinkedIn username or email here
time.sleep(5)

In [ ]:
p=wd.find_element_by_name('session_password')
p.send_keys('Password Input') #Give your LinkedIn password here
time.sleep(5)

In [ ]:
#signin button
wd.find_element_by_xpath('//button[@type="submit"]').click()
time.sleep(5)

#Cosider the following lines only if you don't given the phone number for the given LinkedIn Account
# skip = wd.find_element_by_class_name('secondary-action')
# skip.click()

In [ ]:
#Give the location input to search profiles in that area/region
loc1 = str(input("Enter the location to search people in that area: "))
url = 'https://www.linkedin.com/search/results/people/?keywords='+loc1+'&origin=SWITCH_SEARCH_VERTICAL'

In [ ]:
#Profile URLs Data

profile_urls = []
for i in range(5):
    url = 'https://www.linkedin.com/search/results/people/?keywords='+loc1+'&origin=SWITCH_SEARCH_VERTICAL'+'&page='+str(i+1)
    l=wd.get(url)
    time.sleep(5)
    l=wd.page_source
    soup = bs(l, 'lxml')
    tags = soup.find_all('a')
    temp = 0
    for tag in tags:
        if(tag.get('class') == ['search-result__result-link', 'ember-view']):
            time.sleep(1)
            temp+=1
            if(temp%2==0):
                p_url = 'https://www.linkedin.com/' + tag.get('href')
                profile_urls.append(p_url)
                print(p_url)
                print("<-------------------------------------------------------------------------------------------------->")


In [ ]:
#Scraping in-profile data

p_len = len(profile_urls)
data = {}
data_no = 0
for z in range(p_len):
    profile = profile_urls[z]
    profile_details=wd.get(profile)
    time.sleep(5)
    profile_details=wd.page_source
    details = open('d1.html', 'w', encoding = 'utf8')
    details.write(profile_details)
    details.close()
    r = open('d1.html', 'r+', encoding = 'utf8')
    soup = bs(r)
    r.close()
    try:
        time.sleep(5)
        #name [li tag] class = t-24
        nameLine = soup.find('li', {'class' : 't-24'})
        name = nameLine.get_text().lstrip().rstrip()
    except:
        name = 'N/A'
#_________________________________________________________________________________________________________
    try:
        #position [h2 tag] class = t-18
        positionLine = soup.find('h2', {'class' : 't-18'})
        position = positionLine.get_text().lstrip().rstrip()
    except:
        position = 'N/A'
#_________________________________________________________________________________________________________
    try:
        #address [li tag] class = t-16
        addressLine = soup.find('li', {'class' : "t-16 t-black t-normal inline-block"})
        address = addressLine.get_text().lstrip().rstrip()
    except:
        address = 'N/A'
#_________________________________________________________________________________________________________
    try:
        #connections [span tag] class = t-16
        connectionsLine = soup.find('span', {'class' : 't-16 t-black t-normal'})
        connections = connectionsLine.get_text().lstrip().rstrip()
    except:
        connections = 'N/A'
#_________________________________________________________________________________________________________
    try:
        #skills
        skillsPage = profile + 'skills'
        skills_details=wd.get(skillsPage)
        time.sleep(5)
        skills_details=wd.page_source
        skill_bs = bs(skills_details)
        skillsLine = skill_bs.find_all('p', {'class' : 't-16 t-black t-bold pv1'})
        skills = ''
        for skill in skillsLine:
            skills = skills + " | " + skill.get_text() 
        skills = skills + " | "
    except:
        skills = 'N/A'
#_________________________________________________________________________________________________________
    try:
        email = ''
        #email
        e=get_soup(profile+'detail/contact-info/')
        email=get_email(e).lstrip().rstrip()
    except:
        email = 'N/A'
#_________________________________________________________________________________________________________
    try:
        #interests
        interestsPage = profile + 'detail/interests/companies/'
        interests_details=wd.get(interestsPage)
        time.sleep(5)
        interests_details=wd.page_source
        company_bs = bs(interests_details)
        companyLine = company_bs.find_all('span', {'class' : 'pv-entity__summary-title-text'})
        companies = ''
        for company in companyLine:
            companies = companies + " | " + company.get_text() 
        companies = companies + " | "
    except:
        companies = 'N/A'   
#_________________________________________________________________________________________________________
    data_no += 1
    data[data_no] = [name, email, position, address, connections, skills, companies, profile]


In [ ]:
#Saving the data scraped into a CSV file named as "profile_data.csv"

data_df = pd.DataFrame.from_dict(data, orient='index', columns = ['Name', 'Email ID', 'Position', 'Address', 'Connections Count', 'Skills', 'Companies Interested', 'Profile URL' ])
data_df.to_csv('profile_data.csv')

In [ ]:
#LinkedIn Logout and Webdriver Abort action

l=wd.get('https://www.linkedin.com/m/logout/')
time.sleep(2)
wd.close()